# Condensed rates vs. experiments
## Zhang & Winfree 2009, Dabby 2013, Genot et al. 2011

# Initialization - Global

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from numpy import log10, sqrt
sns.set(style="darkgrid")

In [ ]:
def sema(vect, fig):
    """Rewrite the semantics column into a more compact, human readable version.
    """
    nv = []
    for line in vect:
        v = []
        for (x,y) in fig.pepperargs[line].items():
            if x == 'enumconc': continue
            if x == 'max_complex_size': continue
            if x == 'max_complex_count': continue
            if x == 'max_reaction_count': continue
            if x == 'condensed': 
                z = x if y is True else 'detailed'
            else:
                z = '{}={}'.format(x,y)
            v.append(z)
        nv.append(', '.join(v))
    return nv

## Data generation 1/2

In [ ]:
from zhang2009_rates import data as zhangdata; z09r = zhangdata()
from dabby2013_rates import data as dabbydata; d13r = dabbydata()

Table1 = pd.DataFrame()

In [ ]:
def fill_table(data):
    for fig in data:
        fig.pepperargs['default']['enumconc'] = 'M' # This effects the output file, but not the rate we see in Table1 (which is always in 'M')
        # Show default enumeration parameters:
        display(fig.pepperargs['default'])
        fig.eval() # use default parameters
              
        for df in fig.get_dataframes():
            df['Semantics'] = sema(df['Semantics'], fig) # rewrite semantics to human readable
            df.to_latex(fig.fname + '.tex', index = False, float_format = '{:.3f}'.format)
            df['Name'] = fig.name # Append name field for plotting. 
            display(df)
            yield df

### Zhang and Winfree 2009

In [ ]:
for x in fill_table(z09r):
    Table1 = Table1.append(x)

In [ ]:
%%timeit
tmp = zhangdata()
for fig in tmp:
    fig.eval(verbose = 1)

### Dabby 2013

In [ ]:
for x in fill_table(d13r):
    Table1 = Table1.append(x)

In [ ]:
%%timeit
tmp = dabbydata()
for fig in tmp:
    fig.eval(verbose = 1)

## Plotting (a), (b), (c)

In [ ]:
# Data adjustments:
tmpfig = Table1.copy()
tmpfig['Rate (experiment)'] = log10(tmpfig['Rate (experiment)'])
tmpfig['Rate (calculated)'] = log10(tmpfig['Rate (calculated)'])
#display(tmpfig)

# Figure layout:
g = sns.lmplot(x="Rate (experiment)", y="Rate (calculated)", col="Name", hue='Name', data=tmpfig, sharex = False, sharey = False,
           col_wrap=2, ci=None, truncate=False, palette="muted", height = 3.6, scatter_kws={"s": 20, "alpha": 1})

(mi, ma)=(-2, 8)
plt.xlim(-2.5, 8.5)
plt.ylim(-2.5, 8.5)

for ax in g.axes:
    ax.xaxis.set_major_formatter(plt.FuncFormatter(lambda x,y: "$10^{{ {:.0f} }}$".format(x)))
    ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x,y: "$10^{{ {:.0f} }}$".format(x)))

    ax.plot([mi, ma], [mi, ma], linewidth=3, color='white', zorder=0)
    ax.set(xlabel='Experimental rate constant [/M/s]')
    ax.set(ylabel='Peppercorn rate constant [/M/s]')
    
g.set_titles(row_template="{row_name}", col_template="{col_name}")

plt.savefig('proximal_rates.pdf')
plt.savefig('proximal_rates.svg')

# Genot et al. 2011

In [ ]:
from genot2011 import data as genot2011_data
g11 = genot2011_data()

In [ ]:
Table2 = pd.DataFrame()
for fig in g11:
    display(fig.pepperargs['default'])
    fig.eval(verbose = 0)
    for df in fig.get_dataframes():
        df['Semantics'] = sema(df['Semantics'], fig) # rewrite semantics to human readable
        df['Name']=fig.name
        Table2 = Table2.append(df)

# Let's combine the full analysis into one Table
display(Table2)
Table2.to_latex('Genot2011data.tex', index=False, float_format='{:.3g}'.format)

In [ ]:
%%timeit
g11 = genot2011_data()
for fig in g11:
    fig.eval(verbose = 0, enumprofile = True) # do not simulate or things like that

In [ ]:
tmpfig = Table2.copy()
tmpfig['Time (experiment)'] = log10(tmpfig['Time (experiment)'])
tmpfig['Time (simulation)'] = log10(tmpfig['Time (simulation)'])
#display(tmpfig)

g = sns.lmplot(x = "Time (experiment)", 
               y = "Time (simulation)", 
               hue = 'Name', 
               height = 3.6, #col="Semantics", 
               data = tmpfig, 
               fit_reg = False, 
               markers = ['o', 'x', 'x', 'v', 'v', 'o'], 
               legend = False)

g.fig.set_size_inches(4.3,3.4)

# Adjust plot
(mi, ma)=(0, 5)
plt.ylim(-.5, 8.)

plt.gca().xaxis.set_major_formatter(plt.FuncFormatter(lambda x,y: "$10^{{ {:.0f} }}$".format(x)))
plt.gca().yaxis.set_major_formatter(plt.FuncFormatter(lambda x,y: "$10^{{ {:.0f} }}$".format(x)))

for ax in plt.gcf().get_axes():
    ax.plot([mi, ma], [mi, ma], linewidth=3, color='white',zorder=0)

g.set(title = 'Genot et al. (2011)\nremote-toehold strand displacement')  
g.set(xlabel = 'Experimental completion time [s]')  
g.set(ylabel = 'Peppercorn completion time [s]')
    
legend = plt.legend(ncol = 1, loc = 'upper left', fontsize = 7.8)
frame = legend.get_frame()
frame.set_facecolor('white')

plt.savefig('remote_rates.pdf', bbox_inches='tight')
plt.savefig('remote_rates.svg', bbox_inches='tight')